In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree
# from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

DATA_PATH = '/kaggle/input/jeju-dataset/'

SEED = 42

train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
trade = pd.read_csv(f'{DATA_PATH}international_trade.csv')

In [ ]:
train['name'] = train['ID'].apply(lambda x: x[:6])
test['name'] = test['ID'].apply(lambda x: x[:6])

In [ ]:
train.rename(columns={'supply(kg)': 'supply_kg', 'price(원/kg)': 'price'}, inplace=True)

test.rename(columns={'supply(kg)': 'supply_kg', 'price(원/kg)': 'price'}, inplace=True)


## 시간 Feature

In [ ]:
train.timestamp = pd.to_datetime(train.timestamp)
test.timestamp = pd.to_datetime(test.timestamp)

# train['y-m'] = train.timestamp.dt.strftime('%Y-%m')
# train['y-m-d'] = pd.to_datetime(train['y-m'] + '-01')
# train['unix_timestamp'] = train['timestamp'].apply(lambda x: int(x.timestamp()))
# train['ym_unix_timestamp'] = train['y-m-d'].apply(lambda x: int(x.timestamp()))
# train = train.drop(columns = 'y-m-d')

# test['y-m'] = train.timestamp.dt.strftime('%Y-%m')
# test['y-m-d'] = pd.to_datetime(test['y-m'] + '-01')
# test['unix_timestamp'] = test['timestamp'].apply(lambda x: int(x.timestamp()))
# test['ym_unix_timestamp'] = test['y-m-d'].apply(lambda x: int(x.timestamp()))
# test = test.drop(columns = 'y-m-d')

In [ ]:
train['year'] = train.timestamp.dt.year
train['month'] = train.timestamp.dt.month
train['day'] = train.timestamp.dt.day # 드롭예정
train['day_of_week'] = train.timestamp.dt.dayofweek
train['day_of_year'] = train.timestamp.dt.dayofyear
train['week_of_year'] = train.timestamp.dt.isocalendar().week  # 연중 주간 (ISO 주차 기준)
train['is_weekend'] = train.day_of_week.apply(lambda x: 1 if x >= 5 else 0) 
train['is_sunday'] = train.day_of_week.apply(lambda x: 1 if x == 6 else 0) 

import math

# 연중일 주기 
def cos_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.cos(2 * math.pi * day_of_year / period)

def sin_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.sin(2 * math.pi * day_of_year / period)

train['cos_year'] = train['timestamp'].apply(cos_year)
train['sin_year'] = train['timestamp'].apply(sin_year)

# 요일 주기 (주간)
def cos_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.cos(2 * math.pi * day_of_week / period)

def sin_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.sin(2 * math.pi * day_of_week / period)

train['cos_dow'] = train['timestamp'].apply(cos_weekday)
train['sin_dow'] = train['timestamp'].apply(sin_weekday)

# 연중주 주기 
def cos_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.cos(2 * math.pi * week_of_year / period)

def sin_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.sin(2 * math.pi * week_of_year / period)

train['cos_week_of_year'] = train['timestamp'].apply(cos_week_of_year)
train['sin_week_of_year'] = train['timestamp'].apply(sin_week_of_year)


In [ ]:
test['year'] = test.timestamp.dt.year
test['month'] = test.timestamp.dt.month
test['day'] = test.timestamp.dt.day # 드롭예정
test['day_of_week'] = test.timestamp.dt.dayofweek
test['day_of_year'] = test.timestamp.dt.dayofyear
test['week_of_year'] = test.timestamp.dt.isocalendar().week  # 연중 주간 (ISO 주차 기준)
test['is_weekend'] = test.day_of_week.apply(lambda x: 1 if x >= 5 else 0) 
test['is_sunday'] = test.day_of_week.apply(lambda x: 1 if x == 6 else 0) 

import math

# 연중일 주기 
def cos_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.cos(2 * math.pi * day_of_year / period)

def sin_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.sin(2 * math.pi * day_of_year / period)

test['cos_year'] = test['timestamp'].apply(cos_year)
test['sin_year'] = test['timestamp'].apply(sin_year)

# 요일 주기 (주간)
def cos_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.cos(2 * math.pi * day_of_week / period)

def sin_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.sin(2 * math.pi * day_of_week / period)

test['cos_dow'] = test['timestamp'].apply(cos_weekday)
test['sin_dow'] = test['timestamp'].apply(sin_weekday)

# 연중주 주기 
def cos_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.cos(2 * math.pi * week_of_year / period)

def sin_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.sin(2 * math.pi * week_of_year / period)

test['cos_week_of_year'] = test['timestamp'].apply(cos_week_of_year)
test['sin_week_of_year'] = test['timestamp'].apply(sin_week_of_year)


In [ ]:
time_corr = train.drop(columns=['ID','timestamp','item','corporation','location','name'])

time_corr.corr()

# 드롭할 칼럼 : day, timestamp 관련, year 

## 지역, 법인, 지역 Feature

In [ ]:
train.shape, test.shape

In [ ]:
df = pd.read_csv(f'{DATA_PATH}train.csv')

df_encoded = pd.get_dummies(df, columns=['item', 'location', 'corporation'])
df_encoded.iloc[:,4:] = df_encoded.iloc[:,4:].applymap(lambda x: 1 if x != 0 else 0)

df_encoded.iloc[:,2:].corr()

# corporation : b, c, f 여부 
# item : bc, cb, cr, rd, tg ** 다 써야함 
# location : j, S 여부> 0.04

In [ ]:
# 상품별_월별_요일별
df = pd.read_csv(f'{DATA_PATH}train.csv')
df_features = df.copy()
df_features.rename(columns={'supply(kg)': 'supply_kg', 'price(원/kg)': 'price'}, inplace=True)

df_features['timestamp'] = pd.to_datetime(df_features['timestamp'])
df_features['name'] = df_features['ID'].apply(lambda x: x[:6])
df_features['month'] = df_features['timestamp'].dt.month
df_features['year'] = df_features['timestamp'].dt.year
df_features['dayofyear'] = df_features['timestamp'].dt.dayofyear
df_features['day'] = df_features['timestamp'].dt.day
df_features['day_of_week'] = df_features['timestamp'].dt.dayofweek

obj_cols = ['item', 'corporation', 'location','name']

for col in obj_cols:
    enc = LabelEncoder()
    df_features[col]=enc.fit_transform(df_features[col])

In [ ]:
price_name_y_m_dow_avg

In [ ]:
# Price -> avg 활용 가능하나, 

# 상품(이름)별 연별 월별 요일별 가격
price_name_y_m_dow_avg = df_features.groupby(['name','year','month','day_of_week'])['price'].mean().reset_index()
df_features = df_features.merge(price_name_y_m_dow_avg, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_avg'))

price_name_y_m_dow_median = df_features.groupby(['name','year','month','day_of_week'])['price'].median().reset_index()
df_features = df_features.merge(price_name_y_m_dow_median, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_median'))

# supply 

supply_corp_m_std = df_features.groupby(['corporation','month'])['supply_kg'].std().reset_index()
df_features = df_features.merge(supply_corp_m_std, on = ['corporation','month'],how='left',suffixes=('','_corp_m_std'))

supply_corp_m_max = df_features.groupby(['corporation','month'])['supply_kg'].max().reset_index()
df_features = df_features.merge(supply_corp_m_max, on = ['corporation','month'],how='left',suffixes=('','_corp_m_max'))

df_corr = df_features.drop(columns = ['ID','timestamp'])
tmp = df_corr.corr()



In [ ]:
for col in tmp.columns : 
    tmp[col] = tmp[col].apply(lambda x: "{:.6f}".format(float(x)))

tmp

In [ ]:
tmp.to_csv('jeju_correlation_1108.csv',index=False)

In [ ]:

# train, test에 추가 
price_name_y_m_dow_avg = train.groupby(['name','year','month','day_of_week'])['price'].mean().reset_index()
train = train.merge(price_name_y_m_dow_avg, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_avg'))
test = test.merge(price_name_y_m_dow_avg, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_avg'))

price_name_y_m_dow_median = train.groupby(['name','year','month','day_of_week'])['price'].median().reset_index()
train = train.merge(price_name_y_m_dow_median, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_median'))
test = test.merge(price_name_y_m_dow_median, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_median'))

supply_corp_m_std = train.groupby(['corporation','month'])['supply_kg'].std().reset_index()
train = train.merge(supply_corp_m_std, on = ['corporation','month'],how='left',suffixes=('','_corp_m_std'))
test = test.merge(supply_corp_m_std, on = ['corporation','month'],how='left',suffixes=('','_corp_m_std'))

supply_corp_m_max = train.groupby(['corporation','month'])['supply_kg'].max().reset_index()
train = train.merge(supply_corp_m_max, on = ['corporation','month'],how='left',suffixes=('','_corp_m_max'))
test = test.merge(supply_corp_m_max, on = ['corporation','month'],how='left',suffixes=('','_corp_m_max'))


In [ ]:
# # 폐기 feature 

# price_name_m_dow_avg = df_features.groupby(['name','month','dow'])['price'].mean().reset_index()
# df_features = df_features.merge(price_name_m_dow_avg, on = ['name','month','dow'],how='left',suffixes=('','_name_m_dow_avg'))

# price_name_m_dow_std = df_features.groupby(['name','month','dow'])['price'].std().reset_index()
# df_features = df_features.merge(price_name_m_dow_std, on = ['name','month','dow'],how='left',suffixes=('','_name_m_dow_std'))

# price_name_y_m_dow_std = df_features.groupby(['name','year','month','dow'])['price'].std().reset_index()
# df_features = df_features.merge(price_name_y_m_dow_std, on = ['name','year','month','dow'],how='left',suffixes=('','_name_y_m_dow_std'))

# price_name_y_m_dow_skew = df_features.groupby(['name','year','month','dow'])['price'].skew().reset_index()
# df_features = df_features.merge(price_name_y_m_dow_skew, on = ['name','year','month','dow'],how='left',suffixes=('','_name_y_m_dow_skew'))

# price_name_y_m_dow_min = df_features.groupby(['name','year','month','dow'])['price'].min().reset_index()
# df_features = df_features.merge(price_name_y_m_dow_min, on = ['name','year','month','dow'],how='left',suffixes=('','_name_y_m_dow_min'))

# price_name_y_m_dow_max = df_features.groupby(['name','year','month','dow'])['price'].max().reset_index()
# df_features = df_features.merge(price_name_y_m_dow_max, on = ['name','year','month','dow'],how='left',suffixes=('','_name_y_m_dow_max'))

#supply

# supply_corp_m_avg = df_features.groupby(['corporation','month'])['supply_kg'].mean().reset_index()
# df_features = df_features.merge(supply_corp_m_avg, on = ['corporation','month'],how='left',suffixes=('','_corp_m_avg'))


# supply_corp_m_skew = df_features.groupby(['corporation','month'])['supply_kg'].skew().reset_index()
# df_features = df_features.merge(supply_corp_m_skew, on = ['corporation','month'],how='left',suffixes=('','_corp_m_skew'))

# supply_corp_m_min = df_features.groupby(['corporation','month'])['supply_kg'].min().reset_index()
# df_features = df_features.merge(supply_corp_m_min, on = ['corporation','month'],how='left',suffixes=('','_corp_m_min'))

# supply_corp_m_median = df_features.groupby(['corporation','month'])['supply_kg'].median().reset_index()
# df_features = df_features.merge(supply_corp_m_median, on = ['corporation','month'],how='left',suffixes=('','_corp_m_median'))


# supply_corp_item_m_avg = df_features.groupby(['corporation','item','month'])['supply_kg'].mean().reset_index()
# df_features = df_features.merge(supply_corp_item_m_avg, on = ['corporation','item','month'],how='left',suffixes=('','_corp_item_m_avg'))

# supply_corp_item_m_std = df_features.groupby(['corporation','item','month'])['supply_kg'].std().reset_index()
# df_features = df_features.merge(supply_corp_item_m_std, on = ['corporation','item','month'],how='left',suffixes=('','_corp_item_m_std'))


## 무역 Feature

In [ ]:
trade['기간']= pd.to_datetime(trade['기간'])
trade['year'] = trade['기간'].dt.year
trade['month'] = trade['기간'].dt.month

trade['수출 금액'] = trade['수출 금액']*1000 * 1200 #천달러 단위 -> 환율 1200원으로 계산
trade['수입 금액'] = trade['수입 금액']*1000 * 1200

trade['무역수지'] = trade['수출 금액'] - trade['수입 금액']
trade['수출 단가'] = trade['수출 금액'] / trade['수출 중량']
trade['수입 단가'] = trade['수입 금액'] / trade['수입 중량']

trade = trade.fillna(0)

trade.columns = ['timestamp','item','exp_supply','exp_revenue','imp_supply','imp_revenue','trade_balance','year','month','exp_price','imp_price']
trade.head(5)

In [ ]:
# 방울다다기 양배추 제외 버전 
mask = trade['item'].str.contains('감귤|브로콜리|순무|당근|양배추')
trade_items = trade[mask]

trade_items = trade_items.replace({'꽃양배추와 브로콜리(broccoli)':'BC',
                           '양배추':'CB',
                           '당근':'CR',
                           '감귤':'TG',
                           '순무':'RD'})
cb_extra = trade_items['item'] == '방울다다기 양배추' # 방울다다기 양배추 미포함
trade_items = trade_items[~cb_extra]

trade_items = trade_items.drop(columns = 'timestamp').reset_index().drop(columns = 'index')


trade_items_month = pd.pivot_table(trade_items, index=['item','month'],fill_value=0).reset_index()
trade_items_month = trade_items_month.round(1)

trade_items = pd.pivot_table(trade_items, index=['item','year','month'],fill_value=0).reset_index()
trade_items = trade_items.round(1)
trade_items

# * 평균이랑 + max-min평균

In [ ]:
# df['timestamp'] = pd.to_datetime(df['timestamp'])
# df['year'] = df['timestamp'].dt.year
# df['month'] = df['timestamp'].dt.month


# df_trade = pd.merge(df,trade_items,on=['item','year','month'],how='left')
# df_trade.iloc[:,5:].corr()

In [ ]:
# df_trade_month = pd.merge(df,trade_items_month,on=['item','month'],how='left')
# df_trade.iloc[:,5:].corr()

In [ ]:
# 버전 1 ) item, year, month 기준 채우고 3월 값들은 0으로 대체 

train = pd.merge(train,trade_items,on=['item','year','month'],how='left')
test = pd.merge(test,trade_items,on=['item','year','month'],how='left')

# # 버전 2 ) item, month 기준 채우기

# train_ft = pd.merge(train,trade_items,on=['item','year','month'],how='left')
# test_ft = pd.merge(test,trade_items,on=['item','year','month'],how='left')

## 검증, 예측 

In [ ]:
train_ft = train.copy()
test_ft = test.copy()


train_ft.shape, test_ft.shape

In [ ]:
cat_train = train_ft[['item','corporation','location']]

cat_train_encoded = pd.get_dummies(cat_train)
cat_train_encoded = cat_train_encoded.applymap(lambda x: 1 if x != 0 else 0)

cat_test = test_ft[['item','corporation','location']]

cat_test_encoded = pd.get_dummies(cat_test)
cat_test_encoded = cat_test_encoded.applymap(lambda x: 1 if x != 0 else 0)


train_ft = pd.concat([train_ft,cat_train_encoded],axis=1)
test_ft = pd.concat([test_ft,cat_test_encoded],axis=1)

train_ft.shape, test_ft.shape

In [ ]:
drop_cols = ['ID','timestamp','item','corporation','location','day','name']

train_ft = train_ft.drop(columns = drop_cols)
test_ft = test_ft.drop(columns = drop_cols)

train_ft = train_ft.astype(float).fillna(0)
test_ft = test_ft.astype(float).fillna(0)


train_ft.shape, test_ft.shape

In [ ]:
X = train_ft.drop(columns = ['price','supply_kg'])
y = train_ft['price']

test_X = test_ft

In [ ]:
# Split
from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2, random_state=SEED)

# Calculate the index to split the data
split_index = int(0.8 * len(X))

# Split the data into training and validation sets
X_train, X_valid = X[:split_index], X[split_index:]
y_train, y_valid = y[:split_index], y[split_index:]

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, test_X.shape

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
model = XGBRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

# hold 707.3795681078395
# 뒷부분 Hold : 753.6527649085411
# 모든 feature 추가 : 560.0123072446424 (onehot encoding)
# holdout : 507.71805618161835

In [ ]:
model = LGBMRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

# 뒷부분 Hold : 843.9928276331788
# ym unix 포함 : 847.5193982992201
# 전체 : 726.5042502409534
# 전체 Holdout : 673.5662950313344

In [ ]:
model = RandomForestRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

# 뒷부분 Hold 343.52258075359885
# ym unix 포함 : 
# 모든 피쳐 포함 : 330.62296335942256 (원핫인코딩))

In [ ]:
# def objective_rf(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
#         'max_depth': trial.suggest_int('max_depth', 3, 50),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#         'gamma': trial.suggest_float('gamma', 0, 1),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'subsample': trial.suggest_float('subsample', 0.1, 1),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
#         # 'enable_categorical': True,
#         # 'tree_method': 'hist'  # 다른 트리 메서드 시도
#     }

#     model = RandomForestRegressor(**params)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_valid)
#     rmse = np.sqrt(mean_squared_error(y_valid, preds))

#     return rmse

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators',1, 20),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        # 'enable_categorical': True,
        # 'tree_method': 'hist'  # 다른 트리 메서드 시도
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))

    return rmse


In [ ]:
!pip install optuna

In [ ]:
!pip install --upgrade optuna


In [ ]:
import optuna

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
study.best_params

best_params = study.best_params


In [ ]:
# 795.6023334161574 
# 검증점수 200, 데이콘 점수 1453.5197642204
# parameters_1 = {'n_estimators': 786, 'max_depth': 49, 'learning_rate': 0.01640221809475724, 'gamma': 0.5313872045132086, 'min_child_weight': 7, 'subsample': 0.9493311412280403, 'colsample_bytree': 0.452219535387997, 'reg_alpha': 3.1307325559709165, 'reg_lambda': 5.577393970296672}

#770.1095557900946 
parameters_2 = {'n_estimators': 103, 'max_depth': 9, 'learning_rate': 0.0505258444097842, 'gamma': 0.49458010701295485, 'min_child_weight': 2, 'subsample': 0.9712798544565732, 'colsample_bytree': 0.9863274787845642, 'reg_alpha': 8.554141810651485, 'reg_lambda': 9.46723821368897}

#798.0813850260189 
parameters_3 = {'n_estimators': 125, 'max_depth': 14, 'learning_rate': 0.06865959996206937, 'gamma': 0.4398092182415023, 'min_child_weight': 3, 'subsample': 0.9948546062004352, 'colsample_bytree': 0.8763514869482193, 'reg_alpha': 1.1420800081673188, 'reg_lambda': 3.68073990592226}

# 845.238901560459 
parameters_4 = {'n_estimators': 772, 'max_depth': 11, 'learning_rate': 0.06209451206011337, 'gamma': 0.1541323298249201, 'min_child_weight': 10, 'subsample': 0.8877306745243241, 'colsample_bytree': 0.3922321423322589, 'reg_alpha': 9.295858630218872, 'reg_lambda': 2.4971154174509858}

# 787.0903710150227 
parameters_5 = {'n_estimators': 142, 'max_depth': 16, 'learning_rate': 0.06378369065734353, 'gamma': 0.729761156994088, 'min_child_weight': 4, 'subsample': 0.45241211483905364, 'colsample_bytree': 0.3683340803089096, 'reg_alpha': 5.9597081798126625, 'reg_lambda': 7.221988182934565}

# 805.11072732653
parameters_6 = {'n_estimators': 101, 'max_depth': 29, 'learning_rate': 0.045041020778917994, 'gamma': 0.1449402468887242, 'min_child_weight': 9, 'subsample': 0.28631909493549706, 'colsample_bytree': 0.2499581704397744, 'reg_alpha': 6.349864749345221, 'reg_lambda': 4.491846885944069}

#766.6204305274048 
parameters_7 = {'n_estimators': 123, 'max_depth': 30, 'learning_rate': 0.04658325501174057, 'gamma': 0.5904902958911891, 'min_child_weight': 6, 'subsample': 0.7510534221357206, 'colsample_bytree': 0.5329109319862763, 'reg_alpha': 5.039989492395438, 'reg_lambda': 9.601023667106475}

In [ ]:
preds = []

for i in tqdm(range(2, 7)):
    model = XGBRegressor(**globals()[f'parameters_{i}'])  
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    
    pred = model.predict(test_X)
    preds.append(pred)
    
    print(f"RMSE for model with parameters_{i}: {rmse}")
    print(np.mean(pred))
    break


In [ ]:
train.month = train.month.astype(str)
np.mean(train[train.month == '3']['price'])

In [ ]:
test_X = test_X.rename(columns={'price':'price_name_y_m_dow_avg',
                                'supply_kg':'supply_kg_corp_m_std'})

In [ ]:
pred = model.predict(test_X)
pred

In [ ]:
submission.shape, pred.shape

In [ ]:
pred[pred < 0] = 0

submission['answer'] = pred
submission

In [ ]:
# submission.to_csv(f"{DATA_PATH}submission12_1108.csv", index=False)
submission.to_csv("submission12_1108.csv", index=False)

In [ ]:
# 다양한 기준으로, 다양한 피처를 추가할 것. name 뿐이아니라!


train

# 요소 
# 시간 : 년, 월, 일, 요일, 연중주, 연중일, 시간? or sin cos (주기)? 
# 품목 
# 법인 
# 지역 
# 공급물량
# 수출금액 
# 수출물량
# 수입금액
# 수입물량
# 무역수지 

import itertools

elements = ['year','month','day','dayofweek','dayofyear','weekofyear',
           'item','corporation','location','supply',
           'imp_supply','imp_sales','exp_supply','exp_sales','trade_balance']  # Replace this with your list of elements

# # Generate all combinations of elements from the list
# all_combinations = []
# for r in range(1, len(elements) + 1):
#     combinations_r = list(itertools.combinations(elements, r))
#     all_combinations.extend(combinations_r)

# # Print all combinations
# print(all_combinations)